In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import mutual_info_regression

# Library data visualization
import plotly
import seaborn as sns
import numpy as np
import matplotlib.pylab as pl
import matplotlib as m
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib import pyplot as plt
from IPython.display import SVG

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)



import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-06-22 13:12:02.656496: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 13:12:02.656754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 13:12:02.836995: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv
/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/submission.csv


In [2]:
df= pd.read_csv("/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv")
df

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise,sunset
0,D0001,C001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,10.0,6.0,11.9,2,06:04 AM,07:19 PM
1,D0002,C001,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,10.0,1.0,23.4,1,06:05 AM,07:18 PM
2,D0003,C001,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,06:05 AM,07:18 PM
3,D0004,C001,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,06:06 AM,07:16 PM
4,D0005,C001,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,10.0,1.0,9.0,1,06:07 AM,07:15 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,D2889,C112,17.1,Clear and Sunny,13.3,61,1017.0,0.0,57,3,17.1,10.0,1.0,22.2,1,05:49 AM,05:51 PM
2889,D2890,C112,17.4,NaN,13.0,51,1017.0,0.0,49,0,17.4,10.0,1.0,22.2,1,05:49 AM,05:52 PM
2890,D2891,C112,19.2,NaN,11.5,46,1016.0,0.0,34,0,19.2,10.0,1.0,21.3,2,05:48 AM,05:52 PM
2891,D2892,C112,19.2,NaN,14.4,76,1017.0,0.0,45,2,19.2,10.0,1.0,24.9,2,05:47 AM,05:52 PM


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2893 entries, 0 to 2892
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   day_id                    2893 non-null   object 
 1   city_id                   2893 non-null   object 
 2   temperature_celsius       2893 non-null   float64
 3   condition_text            479 non-null    object 
 4   wind_kph                  2893 non-null   float64
 5   wind_degree               2893 non-null   int64  
 6   pressure_mb               2893 non-null   float64
 7   precip_mm                 2893 non-null   float64
 8   humidity                  2893 non-null   int64  
 9   cloud                     2893 non-null   int64  
 10  feels_like_celsius        2893 non-null   float64
 11  visibility_km             2893 non-null   float64
 12  uv_index                  2893 non-null   float64
 13  gust_kph                  2893 non-null   float64
 14  air_qual

In [4]:
df = df.drop_duplicates()

In [5]:
df.nunique()

day_id                      2893
city_id                      112
temperature_celsius          191
condition_text                 9
wind_kph                      84
wind_degree                  281
pressure_mb                   49
precip_mm                    122
humidity                      94
cloud                         87
feels_like_celsius           394
visibility_km                 30
uv_index                      10
gust_kph                     374
air_quality_us-epa-index       6
sunrise                      168
sunset                       198
dtype: int64

In [6]:
def convert_to_hour(time_str):
    if pd.isna(time_str):
        return None
    time_obj = pd.to_datetime(time_str, format='%I:%M %p')
    return time_obj.hour + time_obj.minute / 60

df['sunrise'] = df['sunrise'].apply(convert_to_hour)
df['sunset'] = df['sunset'].apply(convert_to_hour)


In [7]:
# 1. Day Length
df['day_length'] = df['sunset'] - df['sunrise']

# 2. Wind Power
df['wind_power'] = df['wind_kph'] + df['gust_kph']

# 3. Temperature and Humidity Interaction
df['temp_humidity_interaction'] = (df['temperature_celsius'] * df['humidity']) / 100

# 4. Weather Intensity
df['weather_intensity'] = df['temperature_celsius'] + df['wind_kph'] + df['precip_mm'] + df['humidity'] + df['cloud']

# 5. Wind Chill Factor
df['wind_chill'] = 13.12 + 0.6215 * df['temperature_celsius'] - 11.37 * (df['wind_kph'] ** 0.16) + 0.3965 * df['temperature_celsius'] * (df['wind_kph'] ** 0.16)

# Normalizing temperature, humidity, and wind_kph as examples
min_temp = df['temperature_celsius'].min()
max_temp = df['temperature_celsius'].max()
df['norm_temperature'] = (df['temperature_celsius'] - min_temp) / (max_temp - min_temp)

min_humidity = df['humidity'].min()
max_humidity = df['humidity'].max()
df['norm_humidity'] = (df['humidity'] - min_humidity) / (max_humidity - min_humidity)

min_wind_kph = df['wind_kph'].min()
max_wind_kph = df['wind_kph'].max()
df['norm_wind_kph'] = (df['wind_kph'] - min_wind_kph) / (max_wind_kph - min_wind_kph)

# # Display the first few rows to verify the added features
# print(data.head())


In [8]:
train_df = df[df['condition_text'].notnull()]
train_df

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,...,sunrise,sunset,day_length,wind_power,temp_humidity_interaction,weather_intensity,wind_chill,norm_temperature,norm_humidity,norm_wind_kph
2,D0003,C001,20.0,Light Rain with Thunder,3.6,10,1011.0,4.50,100,75,...,6.083333,19.300000,13.216667,16.2,20.000,203.10,21.327527,0.468085,1.000000,0.000000
3,D0004,C001,17.0,Clear and Sunny,6.1,150,1018.0,0.00,88,0,...,6.100000,19.266667,13.166667,17.3,14.960,111.10,17.502676,0.404255,0.875000,0.035411
6,D0007,C001,21.0,Partly Cloudy,4.0,310,1015.0,0.00,100,50,...,6.133333,19.183333,13.050000,19.1,21.000,175.00,22.372206,0.489362,1.000000,0.005666
18,D0019,C001,19.0,Clear and Sunny,3.6,64,1017.0,0.00,88,0,...,6.333333,18.850000,12.516667,11.9,16.720,110.60,20.219338,0.446809,0.875000,0.000000
27,D0028,C002,19.0,Partly Cloudy,3.6,83,1010.0,0.00,73,25,...,6.283333,19.333333,13.050000,11.9,13.870,120.60,20.219338,0.446809,0.718750,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,D2839,C110,27.0,Partly Cloudy,19.1,90,1014.0,0.01,70,25,...,5.650000,17.716667,12.066667,64.7,18.900,141.11,28.835228,0.617021,0.687500,0.219547
2863,D2864,C111,21.4,Light Precipitation,3.6,178,1012.0,0.00,89,75,...,6.283333,18.416667,12.133333,10.2,19.046,189.00,22.878992,0.497872,0.885417,0.000000
2873,D2874,C112,16.9,Clear and Sunny,16.6,68,1018.0,0.00,44,2,...,6.033333,17.816667,11.783333,43.6,7.436,79.50,16.304293,0.402128,0.416667,0.184136
2886,D2887,C112,19.5,Clear and Sunny,5.4,27,1014.0,0.00,34,0,...,5.850000,17.850000,12.000000,16.6,6.630,58.90,20.474111,0.457447,0.312500,0.025496


In [9]:
condition_texts = train_df.groupby('condition_text')['condition_text'].nunique()
condition_texts

condition_text
Clear and Sunny            1
Cloudy and Overcast        1
Light Precipitation        1
Light Rain with Thunder    1
Mist or Fog                1
Moderate to Heavy Rain     1
Partly Cloudy              1
Rain Showers               1
Thunderstorms              1
Name: condition_text, dtype: int64

In [10]:
train_df['condition_text'] = train_df['condition_text'].replace('Clear and Sunny',1)
train_df['condition_text'] = train_df['condition_text'].replace('Cloudy and Overcast',2)
train_df['condition_text'] = train_df['condition_text'].replace('Rain Showers',3)
train_df['condition_text'] = train_df['condition_text'].replace('Light Precipitation',4)
train_df['condition_text'] = train_df['condition_text'].replace('Light Rain with Thunder',5)
train_df['condition_text'] = train_df['condition_text'].replace('Mist or Fog',6)
train_df['condition_text'] = train_df['condition_text'].replace('Moderate to Heavy Rain',7)
train_df['condition_text'] = train_df['condition_text'].replace('Partly Cloudy',8)
train_df['condition_text'] = train_df['condition_text'].replace('Thunderstorms',9)
train_df

/tmp/ipykernel_18/1247471773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['condition_text'] = train_df['condition_text'].replace('Clear and Sunny',1)
/tmp/ipykernel_18/1247471773.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['condition_text'] = train_df['condition_text'].replace('Cloudy and Overcast',2)
/tmp/ipykernel_18/1247471773.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,...,sunrise,sunset,day_length,wind_power,temp_humidity_interaction,weather_intensity,wind_chill,norm_temperature,norm_humidity,norm_wind_kph
2,D0003,C001,20.0,5,3.6,10,1011.0,4.50,100,75,...,6.083333,19.300000,13.216667,16.2,20.000,203.10,21.327527,0.468085,1.000000,0.000000
3,D0004,C001,17.0,1,6.1,150,1018.0,0.00,88,0,...,6.100000,19.266667,13.166667,17.3,14.960,111.10,17.502676,0.404255,0.875000,0.035411
6,D0007,C001,21.0,8,4.0,310,1015.0,0.00,100,50,...,6.133333,19.183333,13.050000,19.1,21.000,175.00,22.372206,0.489362,1.000000,0.005666
18,D0019,C001,19.0,1,3.6,64,1017.0,0.00,88,0,...,6.333333,18.850000,12.516667,11.9,16.720,110.60,20.219338,0.446809,0.875000,0.000000
27,D0028,C002,19.0,8,3.6,83,1010.0,0.00,73,25,...,6.283333,19.333333,13.050000,11.9,13.870,120.60,20.219338,0.446809,0.718750,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,D2839,C110,27.0,8,19.1,90,1014.0,0.01,70,25,...,5.650000,17.716667,12.066667,64.7,18.900,141.11,28.835228,0.617021,0.687500,0.219547
2863,D2864,C111,21.4,4,3.6,178,1012.0,0.00,89,75,...,6.283333,18.416667,12.133333,10.2,19.046,189.00,22.878992,0.497872,0.885417,0.000000
2873,D2874,C112,16.9,1,16.6,68,1018.0,0.00,44,2,...,6.033333,17.816667,11.783333,43.6,7.436,79.50,16.304293,0.402128,0.416667,0.184136
2886,D2887,C112,19.5,1,5.4,27,1014.0,0.00,34,0,...,5.850000,17.850000,12.000000,16.6,6.630,58.90,20.474111,0.457447,0.312500,0.025496


In [11]:
# Remove 'D' from `day_id` and 'C' from `city_id`
train_df['day_id'] = train_df['day_id'].astype(str).str.replace('D', '', regex=False)
train_df['city_id'] = train_df['city_id'].astype(str).str.replace('C', '', regex=False)

# Convert `day_id` and `city_id` to integers
train_df['day_id'] = pd.to_numeric(train_df['day_id'])
train_df['city_id'] = pd.to_numeric(train_df['city_id'])
train_df['day_id'] = train_df['day_id'].astype(int)
train_df['city_id'] = train_df['city_id'].astype(int)
train_df['condition_text'].astype(str).astype(int)

train_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 479 entries, 2 to 2888
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   day_id                     479 non-null    int64  
 1   city_id                    479 non-null    int64  
 2   temperature_celsius        479 non-null    float64
 3   condition_text             479 non-null    int64  
 4   wind_kph                   479 non-null    float64
 5   wind_degree                479 non-null    int64  
 6   pressure_mb                479 non-null    float64
 7   precip_mm                  479 non-null    float64
 8   humidity                   479 non-null    int64  
 9   cloud                      479 non-null    int64  
 10  feels_like_celsius         479 non-null    float64
 11  visibility_km              479 non-null    float64
 12  uv_index                   479 non-null    float64
 13  gust_kph                   479 non-null    float64
 14

/tmp/ipykernel_18/1674010490.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['day_id'] = train_df['day_id'].astype(str).str.replace('D', '', regex=False)
/tmp/ipykernel_18/1674010490.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['city_id'] = train_df['city_id'].astype(str).str.replace('C', '', regex=False)
/tmp/ipykernel_18/1674010490.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [12]:
# label = 'condition_text' 
# X = train_df.drop(columns=[label], axis=1)  
# y = train_df[label]

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5, test_size=0.001)

# train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(pd.concat([X_train, y_train], axis=1), label=label, task=tfdf.keras.Task.CLASSIFICATION)
# test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(pd.concat([X_test, y_test], axis=1), label=label, task=tfdf.keras.Task.CLASSIFICATION)

# # Improved Model with Hyperparameter Tuning
# model1 = tfdf.keras.GradientBoostedTreesModel(
#     task=tfdf.keras.Task.CLASSIFICATION,
#     num_trees=500,       # Increased trees for potential improvement
#     growing_strategy="BEST_FIRST_GLOBAL",  # Explore alternative growing strategies
#     max_depth=25,        # Tune this based on your data complexity
#     l1_regularization=0.01,   # Add regularization to prevent overfitting
#     l2_regularization=0.01,
#     min_examples=2,     # Tune this
#     shrinkage=0.1,       # Learning rate
#     subsample=0.8,     # Add subsampling for potential improvement
#     categorical_algorithm="RANDOM",  # Consider "HISTOGRAM" as well
#     random_seed=5 
# )

# model1.compile(metrics=["accuracy"]) 

# model1.fit(x=train_ds, validation_data=test_ds)

# evaluation = model1.evaluate(test_ds, return_dict=True)

# y_pred = model1.predict(test_ds)

# y_pred_rounded = tf.cast(tf.round(y_pred), tf.int32) 

# y_pred = tf.argmax(y_pred_rounded, axis=1)

# from sklearn.metrics import accuracy_score
# print('*'*20)
# print('Accuracy  :',np.round(accuracy_score(y_test, y_pred)*100,2),'%')       #  72.22 %
# print('*'*20)







# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.ensemble import RandomForestClassifier
# import tensorflow_decision_forests as tfdf
# import tensorflow as tf

# Assuming train_df is your DataFrame
label = 'condition_text'

# Feature Engineering
train_df[label] = train_df[label].astype(str)

# Use LabelEncoder to ensure positive integer labels
le = LabelEncoder()
train_df['label_numeric'] = le.fit_transform(train_df[label]) + 1  # Add 1 to ensure positive labels

# Normalize numerical features
scaler = StandardScaler()
numerical_features = train_df.select_dtypes(include=[np.number]).columns.drop('label_numeric')
train_df[numerical_features] = scaler.fit_transform(train_df[numerical_features])

# Split the data
X = train_df.drop(columns=[label, 'label_numeric'], axis=1)
y = train_df['label_numeric']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5, test_size=0.1)

# Convert to TensorFlow datasets
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(pd.concat([X_train, y_train], axis=1), label='label_numeric', task=tfdf.keras.Task.CLASSIFICATION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(pd.concat([X_test, y_test], axis=1), label='label_numeric', task=tfdf.keras.Task.CLASSIFICATION)





# Try another model (Random Forest) for comparison
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    max_depth=None,
    n_estimators=100,
    min_samples_split=5,
    min_samples_leaf=1,
    max_features='log2'
)

rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

print('Random Forest Accuracy:', np.round(accuracy_score(y_test, rf_pred)*100, 2), '%')
print('*'*20)

/tmp/ipykernel_18/2717432232.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[label] = train_df[label].astype(str)
/tmp/ipykernel_18/2717432232.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['label_numeric'] = le.fit_transform(train_df[label]) + 1  # Add 1 to ensure positive labels
/tmp/ipykernel_18/2717432232.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

NameError: name 'accuracy_score' is not defined

In [ ]:
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [None, 10, 20],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['auto', 'sqrt', 'log2']
# }
# from sklearn.model_selection import GridSearchCV

# grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=5),
#                            param_grid=param_grid,
#                            cv=5,
#                            scoring='accuracy',
#                            n_jobs=-1)
# grid_search.fit(X_train, y_train)

# print("Best parameters found: ", grid_search.best_params_)
# print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.ensemble import RandomForestClassifier
# import tensorflow_decision_forests as tfdf
# import tensorflow as tf

# # Assuming train_df is your DataFrame
# label = 'condition_text'

# # Feature Engineering
# train_df[label] = train_df[label].astype(str)

# # Use LabelEncoder to ensure positive integer labels
# # le = LabelEncoder()
# # train_df['label_numeric'] = le.fit_transform(train_df[label]) + 1  # Add 1 to ensure positive labels

# # # Normalize numerical features
# # scaler = StandardScaler()
# # numerical_features = train_df.select_dtypes(include=[np.number]).columns.drop('label_numeric')
# # train_df[numerical_features] = scaler.fit_transform(train_df[numerical_features])

# # Split the data
# X = train_df.drop(columns=[label], axis=1)
# y = train_df[label]
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5, test_size=0.1)

# # Try another model (Random Forest) for comparison
# rf_model = RandomForestClassifier(n_estimators=100, random_state=5)
# rf_model.fit(X_train, y_train)
# rf_pred = rf_model.predict(X_test)

# print('Random Forest Accuracy:', np.round(accuracy_score(y_test, rf_pred)*100, 2), '%')
# print('*'*20)



In [ ]:
df['condition_text'] = df['condition_text'].replace('Clear and Sunny',1)
df['condition_text'] = df['condition_text'].replace('Cloudy and Overcast',2)
df['condition_text'] = df['condition_text'].replace('Rain Showers',3)
df['condition_text'] = df['condition_text'].replace('Light Precipitation',4)
df['condition_text'] = df['condition_text'].replace('Light Rain with Thunder',5)
df['condition_text'] = df['condition_text'].replace('Mist or Fog',6)
df['condition_text'] = df['condition_text'].replace('Moderate to Heavy Rain',7)
df['condition_text'] = df['condition_text'].replace('Partly Cloudy',8)
df['condition_text'] = df['condition_text'].replace('Thunderstorms',9)
copy_df = df
df

In [ ]:
# Remove 'D' from `day_id` and 'C' from `city_id`
df['day_id'] = df['day_id'].astype(str).str.replace('D', '', regex=False)
df['city_id'] = df['city_id'].astype(str).str.replace('C', '', regex=False)

# Convert `day_id` and `city_id` to integers
df['day_id'] = pd.to_numeric(df['day_id'])
df['city_id'] = pd.to_numeric(df['city_id'])

df['day_id'] = df['day_id'].astype(int)
df['city_id'] = df['city_id'].astype(int)


In [ ]:
test_df = df[df.isnull().any(axis=1)]
test_df

In [ ]:
test_df.info()

In [ ]:
test_df = test_df.drop(columns=['condition_text'])

In [ ]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    test_df,
    task=tfdf.keras.Task.CLASSIFICATION
)

In [ ]:
rf_pred = model1.predict(test_ds)


In [ ]:
rf_pred.shape

In [ ]:
rf_pred_rounded = tf.cast(tf.round(rf_pred), tf.int32) 

rf_pred = tf.argmax(rf_pred_rounded, axis=1)

In [ ]:
test_df['condition_text'] = rf_pred

In [ ]:
test_df.info()

In [ ]:
# new_df = pd.read_csv("")

In [ ]:
# copy_df = copy_df.drop(columns=['temperature_celsius','wind_kph','wind_degree','pressure_mb','precip_mm','humidity','cloud','feels_like_celsius','visibility_km','uv_index','gust_kph','air_quality_us-epa-index','sunrise','sunset'], axis=1)
copy_df = copy_df[['day_id', 'city_id','condition_text']]

In [ ]:

copy_df = copy_df.merge(test_df[['day_id', 'city_id', 'condition_text']], on=['day_id', 'city_id'], how='left')

# Create the new column 'condition_text'
copy_df['condition_text'] = copy_df['condition_text_x'].combine_first(copy_df['condition_text_y'])

# Optionally, you can drop the old columns if they are no longer needed
copy_df = copy_df.drop(columns=['condition_text_x', 'condition_text_y'])


In [ ]:
copy_df['condition_text'].value_counts()

In [ ]:
submission_df = pd.read_csv("/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/submission.csv")
submission_df['condition_text'] = copy_df['condition_text']


In [ ]:
submission_df['condition_text'].value_counts()

In [ ]:
submission_df['condition_text'] = submission_df['condition_text'].replace({
    1: 'Clear and Sunny',
    2: 'Cloudy and Overcast',
    3: 'Rain Showers',
    4: 'Light Precipitation',
    5: 'Light Rain with Thunder',
    6: 'Mist or Fog',
    7: 'Moderate to Heavy Rain',
    8: 'Partly Cloudy',
    9: 'Thunderstorms',
    0:'Clear and Sunny'
})

In [ ]:
submission_df['condition_text'].value_counts()

In [ ]:
final = submission_df
final.to_csv('prediction_6.csv' , index=False)
